<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/CRYPTO_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ccxt -q

In [7]:
import ccxt
import time
import numpy as np
from sklearn.linear_model import LogisticRegression
from datetime import datetime, timedelta

# --- Public API Functions ---
def get_exchange_client(exchange_id):
    """
    Creates and returns a ccxt exchange client instance for public API calls.
    """
    try:
        exchange_class = getattr(ccxt, exchange_id)
        exchange = exchange_class({})
        return exchange
    except AttributeError:
        print(f"Error: Exchange '{exchange_id}' not found in ccxt.")
        return None
    except Exception as e:
        return {"error": f"Error creating exchange client: {e}"}

def fetch_public_prices(exchange, symbols):
    """
    Fetches the current ticker price for a list of symbols.
    """
    try:
        tickers = exchange.fetch_tickers(symbols)
        prices = {}
        for symbol in symbols:
            if symbol in tickers:
                prices[symbol] = tickers[symbol]['last']
            else:
                prices[symbol] = "N/A (Symbol not found or inactive)"
        return prices
    except ccxt.NetworkError as e:
        return {"error": f"A network error occurred: {e}"}
    except ccxt.ExchangeError as e:
        return {"error": f"An exchange error occurred: {e}"}
    except Exception as e:
        return {"error": f"An unexpected error occurred: {e}"}

def fetch_historical_ohlcv(exchange, symbol, timeframe='1h', limit=100):
    """
    Fetches historical OHLCV data for a given symbol and timeframe.
    """
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        return ohlcv
    except ccxt.NetworkError as e:
        print(f"Network error fetching historical data: {e}")
        return None
    except ccxt.ExchangeError as e:
        print(f"Exchange error fetching historical data: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error fetching historical data: {e}")
        return None

# --- Machine Learning Model POC ---
def create_ml_model(historical_data):
    """
    Uses historical data to create and train a simple machine learning model.
    """
    if not historical_data or len(historical_data) < 2:
        print("Not enough historical data to train the model.")
        return None, None

    close_prices = np.array([candle[4] for candle in historical_data])
    X = close_prices[:-1].reshape(-1, 1)
    y = (close_prices[1:] > close_prices[:-1]).astype(int)

    model = LogisticRegression()
    model.fit(X, y)

    last_close_price = close_prices[-1]

    return model, last_close_price

def predict_with_model(model, last_known_price, current_price):
    """
    Uses the trained model to make a simple prediction.
    """
    input_data = np.array([[current_price]])
    prediction = model.predict(input_data)[0]

    if prediction == 1:
        return "rise", "The model predicts the next price will be higher than the last known price."
    else:
        return "fall", "The model predicts the next price will be lower than or equal to the last known price."

# --- Agentic Component ---
def make_agentic_decision(prediction_label):
    """
    A simple agent that makes a trading decision based on the model's prediction.
    NOTE: This is a simulated decision for a POC and does not execute real trades.

    Args:
        prediction_label (str): The 'rise' or 'fall' label from the model.

    Returns:
        str: A simulated trading action ('BUY', 'SELL', or 'HOLD').
    """
    if prediction_label == "rise":
        return "BUY"
    elif prediction_label == "fall":
        return "SELL"
    else:
        return "HOLD"

if __name__ == "__main__":
    exchange_id = 'kraken'

    # --- Part 1: Fetching Historical Data and Training the Model ---
    print("--- Part 1: Fetching Historical Data and Training the Model ---")

    kraken_client = get_exchange_client(exchange_id)
    if not kraken_client:
        exit()

    ml_symbol = 'BTC/USD'
    print(f"Fetching historical data for {ml_symbol} to train the model...")
    historical_data = fetch_historical_ohlcv(kraken_client, ml_symbol, timeframe='1h', limit=100)

    ml_model, last_close_price = create_ml_model(historical_data)

    if not ml_model:
        print("Failed to create the machine learning model.")
        exit()
    else:
        print("Model trained successfully on historical data.")

    # --- Part 2: Fetching Live Prices and Making a Prediction ---
    print("\n\n--- Part 2: Fetching Live Prices and Making a Prediction ---")

    trading_symbols = ['BTC/USD', 'ETH/USD', 'SOL/USD', 'TAO/USD']
    print(f"Fetching live prices from {exchange_id.upper()} for {', '.join(trading_symbols)}...")

    prices_data = fetch_public_prices(kraken_client, trading_symbols)

    if "error" in prices_data:
        print(prices_data["error"])
        exit()
    else:
        for symbol, price in prices_data.items():
            print(f"Current price of {symbol}: ${price}")

    # --- Part 3: Agentic Decision-Making ---
    print("\n\n--- Part 3: Agentic Decision-Making ---")

    current_btc_price = prices_data.get('BTC/USD')
    if current_btc_price and last_close_price:
        prediction_label, prediction_text = predict_with_model(ml_model, last_close_price, current_btc_price)

        print(f"Last historical close price: ${last_close_price}")
        print(f"Current live price:          ${current_btc_price}")
        print(f"Model Prediction:            {prediction_text}")

        # The agent makes a simulated decision based on the prediction
        agent_action = make_agentic_decision(prediction_label)

        print(f"Agent's Simulated Action:    {agent_action}")
        print("\nNOTE: This is a simulated action for a Proof of Concept and does not execute real trades.")
    else:
        print("\nError: Could not get a valid BTC price to run the agent.")

--- Part 1: Fetching Historical Data and Training the Model ---
Fetching historical data for BTC/USD to train the model...
Model trained successfully on historical data.


--- Part 2: Fetching Live Prices and Making a Prediction ---
Fetching live prices from KRAKEN for BTC/USD, ETH/USD, SOL/USD, TAO/USD...
Current price of BTC/USD: $113175.8
Current price of ETH/USD: $3478.99
Current price of SOL/USD: $163.02
Current price of TAO/USD: $335.8775


--- Part 3: Agentic Decision-Making ---
Last historical close price: $113225.0
Current live price:          $113175.8
Model Prediction:            The model predicts the next price will be lower than or equal to the last known price.
Agent's Simulated Action:    SELL

NOTE: This is a simulated action for a Proof of Concept and does not execute real trades.
